In [ ]:
!git clone "https://github.com/Skuldur/Classical-Piano-Composer"

Cloning into 'Classical-Piano-Composer'...
remote: Enumerating objects: 334, done.
remote: Total 334 (delta 0), reused 0 (delta 0), pack-reused 334
Receiving objects: 100% (334/334), 721.79 MiB | 21.78 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [ ]:
!pip install music21

In [ ]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord,stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [ ]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [ ]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("/content/midi_files/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            #print(s2)
            notes_to_parse = s2.parts[0].recurse()
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        #print(notes_to_parse)
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/Classical-Piano-Composer/data/notes1', 'wb') as filepath:
        pickle.dump(notes, filepath)
    print(notes)
    return notes

In [ ]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    #print(note_to_int)
    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [ ]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [ ]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "/content/Classical-Piano-Composer/weights2.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=50, batch_size=128, callbacks=callbacks_list)

In [ ]:
notes = get_notes()

Parsing /content/midi_files/Guru-Mayya Mayya.mp3.mid
Parsing /content/midi_files/Chor Chor-Chandralekha.mp3.mid
Parsing /content/midi_files/Guru-Barso Re.Mp3.mid
Parsing /content/midi_files/Dil se-Jiya Jale.mp3.mid
['4.7', 'B4', 'G4', 'E4', 'B3', 'B4', 'G4', '4.6', 'B4', 'E4', 'G4', 'B4', 'F#6', 'G4', 'F#6', 'G4', 'F#4', '11.4', 'G4', '6.11', 'E4', 'B4', 'E4', 'E4', 'F#6', 'G4', '7.11', 'B4', 'E4', 'G4', 'F#6', 'B4', 'B3', 'G4', 'G4', 'B4', 'B4', 'B4', 'G4', 'B4', 'G4', 'F#6', 'E4', 'G4', 'F#6', 'E4', 'B3', 'G4', 'B4', 'B4', 'E4', 'G4', 'G4', '11.4', 'F#4', 'B4', 'G4', 'E4', '6.7', 'G4', 'E4', 'B6', 'E4', 'B4', 'B4', 'B3', 'E4', '7.11', 'F#6', 'G4', 'B4', 'E3', 'E4', 'G4', 'B4', 'E4', 'G4', 'B4', 'B4', 'B4', 'E4', 'G4', 'E4', 'F#4', 'G4', '11', 'E4', 'E4', 'B5', 'B4', 'G4', 'B4', 'E4', 'G4', '6.11', 'G4', 'B4', '6.7', 'G4', 'E4', 'G4', 'G4', 'B4', 'G4', '6.11', '11.4', 'E4', 'B4', 'G4', 'F#4', 'E4', 'G4', 'B4', 'G4', 'G4', 'F#4', 'E4', 'G4', 'E5', 'E4', 'B4', 'G4', 'B3', 'B4', 'E4', 'B

In [ ]:
n_vocab=len(set(notes))

In [ ]:
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
len(network_output[1])

3439

In [ ]:
network_input

array([[[0.56935156],
        [0.98022681],
        [0.99883687],
        ...,
        [0.67839488],
        [0.99883687],
        [0.99011341]],

       [[0.98022681],
        [0.99883687],
        [0.99011341],
        ...,
        [0.99883687],
        [0.99011341],
        [0.99883687]],

       [[0.99883687],
        [0.99011341],
        [0.97993603],
        ...,
        [0.99011341],
        [0.99883687],
        [0.99883687]],

       ...,

       [[0.67635941],
        [0.99040419],
        [0.25239895],
        ...,
        [0.99011341],
        [0.97993603],
        [0.98953184]],

       [[0.99040419],
        [0.25239895],
        [0.97993603],
        ...,
        [0.97993603],
        [0.98953184],
        [0.99011341]],

       [[0.25239895],
        [0.97993603],
        [0.99011341],
        ...,
        [0.98953184],
        [0.99011341],
        [0.98953184]]])

In [ ]:

model = create_network(network_input, n_vocab)


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_1 (Activation)    (None, 256)              

In [ ]:
train(model, network_input, network_output)

Epoch 1/50
50702/50702 [==============================] - 217s 4ms/step - loss: 7.1208
Epoch 2/50
50702/50702 [==============================] - 215s 4ms/step - loss: 6.6846
Epoch 3/50
50702/50702 [==============================] - 215s 4ms/step - loss: 6.6796
Epoch 4/50
50702/50702 [==============================] - 208s 4ms/step - loss: 6.6321
Epoch 5/50
50702/50702 [==============================] - 202s 4ms/step - loss: 6.5910
Epoch 6/50
50702/50702 [==============================] - 202s 4ms/step - loss: 6.5660
Epoch 7/50
50702/50702 [==============================] - 202s 4ms/step - loss: 6.5268
Epoch 8/50
50702/50702 [==============================] - 201s 4ms/step - loss: 6.4902
Epoch 9/50
50702/50702 [==============================] - 206s 4ms/step - loss: 6.4558
Epoch 10/50
50702/50702 [==============================] - 218s 4ms/step - loss: 6.4318
Epoch 11/50
50702/50702 [==============================] - 209s 4ms/step - loss: 6.4341
Epoch 12/50
50702/50702 [================

In [ ]:
def prepare_sequences1(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [ ]:
def create_network1(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('/content/Classical-Piano-Composer/weights2.hdf5')

    return model

In [ ]:
def generate_notes1(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [ ]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

In [ ]:

#load the notes used to train the model
with open('/content/Classical-Piano-Composer/data/notes1', 'rb') as filepath:
    notes = pickle.load(filepath)

# Get all pitch names
pitchnames = sorted(set(item for item in notes))
# Get all pitch names
n_vocab = len(set(notes))

network_input, normalized_input = prepare_sequences1(notes, pitchnames, n_vocab)
model = create_network1(normalized_input, n_vocab)
prediction_output = generate_notes1(model, network_input, pitchnames, n_vocab)
create_midi(prediction_output)